<a href="https://colab.research.google.com/github/chulwoojung/cwjung-ml/blob/master/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 필요 라이브러리 및 함수 불러오기
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0, '/content/gdrive/My Drive/SKT_Lab')
from utils.load_data import data_loading
from utils.load_data import get_titles
from utils.evaluation import evaluation

In [0]:
# 데이터 불러오기
train_matrix, test_matrix, num_users, num_items = data_loading()

# 하이퍼 파라미터 설정
hidden_dim = 30
learning_rate = 0.01
training_epochs = 10 # 총 4번 학습


 train_ml-100k
max user id 942
max item id 1681

 test_ml_100k
max user id 942
max item id 1670


In [0]:
# tensorflow를 이용하여, AE 모델 만들기

# 플레이스홀더 구성
X = tf.placeholder("float", [None, num_items])



In [0]:
# 파라미터 초기화
encoder_w = tf.get_variable("encoder_w", shape=[num_items, hidden_dim], initializer=tf.contrib.layers.xavier_initializer())
encoder_b = tf.Variable(name="encoder_b", initial_value=tf.zeros([hidden_dim]))

decoder_w = tf.get_variable("decoder_w", shape=[hidden_dim, num_items], initializer=tf.contrib.layers.xavier_initializer())
decoder_b = tf.Variable(name="decoder_b", initial_value=tf.zeros([num_items]))

ValueError: ignored

In [0]:
# AE 만들기
# encoder
hidden_layer = tf.nn.sigmoid(tf.matmul(X, encoder_w) + encoder_b)

# decoder
output = tf.nn.sigmoid(tf.matmul(hidden_layer, decoder_w) + decoder_b)

In [0]:
# 예측 값과 실제 값의 차이 설정
loss = -tf.reduce_sum(X * tf.log(output + 1e-10) + (1 - X) * tf.log(1 - output + 1e-10) )

# 학습 방법 설정(gradient descent방법 설정)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [0]:
# 세션 생성 및 파라미터 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 파라미터 초기화

In [0]:
# 모델 학습
for epoch in range(training_epochs):
    avg_loss = 0

    # 한 명의 사용자 시청 정보를 이용하여 AE를 학습 함
    for u_id in range(num_users):
        _, l = sess.run([optimizer, loss], feed_dict={X: [train_matrix[u_id]]})
        avg_loss += l
    avg_loss /= num_users

    print(epoch, "loss=", avg_loss)

0 loss= 329.1861190876804
1 loss= 280.52104421637847
2 loss= 256.39309614308945
3 loss= 241.39226531577893
4 loss= 228.55371985329296
5 loss= 218.3394885745812
6 loss= 208.49026439096363
7 loss= 202.41224367823352
8 loss= 197.66986409629263
9 loss= 193.48405137168263


In [0]:
# AE를 이용하여 전체 항목의 값을 예측함
predicted_matrix = np.zeros((num_users, num_items))
for u_id in range(0, num_users):
    predicted_matrix[u_id] = sess.run(output, feed_dict={X: [train_matrix[u_id]]})[0]

In [0]:
predicted_matrix[100]

array([0.52115071, 0.14270073, 0.08510602, ..., 0.00117478, 0.00120598,
       0.00118517])

In [0]:
# 평가
hit50, hit100 = evaluation(predicted_matrix, train_matrix, test_matrix)
print("Hit@50:  ", hit50)
print("Hit@100: ", hit100)

Hit@50:   0.2014846235418876
Hit@100:  0.3064687168610817


In [0]:
# 시청 항목과 추천 항목 출력
print("-----------------------------------------------------")
u_id = 201
titles = get_titles()

print("사용자 시청 영화")
rated_items = np.where(train_matrix[u_id] == 1)[0]
rated_items = rated_items[0:20]
for i in rated_items:
    print(titles[i])

print("-----------------------------------------------------")

print("추천 영화 Top 20")
predicted_matrix[u_id, rated_items] = 0
topN_items = np.argsort(predicted_matrix[u_id])
topN_items = topN_items[::-1]
topN_items = topN_items[0:20]
for i in topN_items:
    print(titles[i])

-----------------------------------------------------
사용자 시청 영화
Toy Story (1995)
Terminator 2: Judgment Day (1991)
empire Strikes Back, The (1980)
Princess Bride, The (1987)
Amadeus (1984)
Terminator, The (1984)
Back to the Future (1985)
Kolya (1996)
Contact (1997)
Full Monty, The (1997)
emma (1996)
english Patient, The (1996)
Schindler's List (1993)
e.T. the extra-Terrestrial (1982)
Apartment, The (1960)
Maltese Falcon, The (1941)
Boot, Das (1981)
Local Hero (1983)
It Happened One Night (1934)
-----------------------------------------------------
추천 영화 Top 20
Toy Story (1995)
Maltese Falcon, The (1941)
empire Strikes Back, The (1980)
Kolya (1996)
Amadeus (1984)
Contact (1997)
Terminator, The (1984)
Full Monty, The (1997)
e.T. the extra-Terrestrial (1982)
Apartment, The (1960)
emma (1996)
english Patient, The (1996)
Back to the Future (1985)
Boot, Das (1981)
Local Hero (1983)
Schindler's List (1993)
Terminator 2: Judgment Day (1991)
It Happened One Night (1934)
Princess Bride, The (198